Chúng ta sẽ tiến hành bài toán cơ bản nhất mà ai cũng bắt đầu khi mới làm quen về việc thiết kế một mô hình CV. Đó là phân loại chữ số viết tay với bộ dữ liệu mnist.

mnist là bộ dữ liệu viết tay gồm 10 loại số, được đánh nhãn từ 1 đến 10, với khoảng 70 ngàn ảnh, chúng ta sẽ chia 60 ngàn ảnh cho huấn luyện và 10 ngàn ảnh cho kiểm tra.

Chúng ta sẽ tiến hành xây dựng mô hình theo kiến trúc như thế này

<td> <img src="https://pic1.zhimg.com/v2-c0bb3e9195753191330faebe77417594_1440w.jpg?source=172ae18b" alt="Drawing" style="width: 30%;"/> </td>




In [76]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model

import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.3.0'

In [13]:
# Kiểm tra GPU đang chạy là gì
! nvidia-smi

Tue Aug 25 07:56:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [95]:
# Định nghĩa một vài thông số
num_classes = 10
epochs = 12
batch_size = 128
lr = 1e-3

In [6]:
# Tải bộ dữ liệu mnist
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [14]:
# Kiểm tra kích thước dữ liệu
train_images.shape
test_images.shape

(10000, 28, 28)

In [12]:
# Chuẩn hóa dữ liệu bằng cách chia cho 255
train_images, test_images = train_images/255.0, test_images/255.0

In [85]:
# Xây dựng kiến trúc mô hình phân loại

class my_model(Model):
  def __init__(self):
    super(my_model, self).__init__()
    self.conv1 = Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28))
    self.pool = MaxPool2D(pool_size=2)
    self.conv2 = Conv2D(32, kernel_size=5, activation='relu')
    self.drop = Dropout(0.25)
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(num_classes, activation='softmax')

  def call(self, x):
    x = tf.reshape(x, [-1, 28, 28, 1])
    x = self.conv1(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = self.pool(x)
    x = self.drop(x)

    x = self.flatten(x)
    x = self.d1(x)
    x = self.drop(x)

    out = self.d2(x)
    return out

In [96]:
model = my_model()
model.compile(optimizer=tf.optimizers.Adam(lr),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [97]:
model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size)
model.evaluate(test_images, test_labels, verbose=2)

Epoch 1/12
469/469 [==============================] - 3s 7ms/step - loss: 0.2995 - accuracy: 0.9069
Epoch 2/12
469/469 [==============================] - 3s 6ms/step - loss: 0.0928 - accuracy: 0.9719
Epoch 3/12
469/469 [==============================] - 3s 6ms/step - loss: 0.0664 - accuracy: 0.9789
Epoch 4/12
469/469 [==============================] - 3s 6ms/step - loss: 0.0533 - accuracy: 0.9834
Epoch 5/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0464 - accuracy: 0.9854
Epoch 6/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0397 - accuracy: 0.9880
Epoch 7/12
469/469 [==============================] - 3s 6ms/step - loss: 0.0379 - accuracy: 0.9879
Epoch 8/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0341 - accuracy: 0.9889
Epoch 9/12
469/469 [==============================] - 3s 6ms/step - loss: 0.0308 - accuracy: 0.9904
Epoch 10/12
469/469 [==============================] - 3s 7ms/step - loss: 0.0275 - accuracy: 0.9913

[0.023816920816898346, 0.9921000003814697]